##Custom Dataset Annotator

####Use the below link to annotate the custom dataset before training.

https://tecoholic.github.io/ner-annotator/

[Annotator tool](https://tecoholic.github.io/ner-annotator/)

In [ ]:
import pandas as pd
import json
import os
# os.chdir(r'C:\Users\deepa\Downloads')

with open('json_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
data

In [ ]:
import pandas as pd
import json
import os


# with open('Cust_Sample_till_20000.json', 'r') as f:
#     data = json.load(f)
# data

train_data = data.copy()
train_data = [tuple(i) for i in train_data]
train_data = [i for n, i in enumerate(train_data) if i not in train_data[:n]]
train_data

for i in train_data:
#     print((i[1]))
    for j in (range(len(i[1]['entities']))):
        i[1]['entities'][j] = tuple(i[1]['entities'][j])
train_data

###Load and Unzip the Pretrained Model

In [1]:
#Unzipping the spacy model folder
!unzip /content/Spacy_CUST_Model_with_emails.zip

Archive:  /content/Spacy_CUST_Model_with_emails.zip
   creating: Spacy_Start_CUST_Model_with_emails_V2.08/attribute_ruler/
  inflating: Spacy_Start_CUST_Model_with_emails_V2.08/attribute_ruler/patterns  
  inflating: Spacy_Start_CUST_Model_with_emails_V2.08/config.cfg  
   creating: Spacy_Start_CUST_Model_with_emails_V2.08/lemmatizer/
   creating: Spacy_Start_CUST_Model_with_emails_V2.08/lemmatizer/lookups/
  inflating: Spacy_Start_CUST_Model_with_emails_V2.08/lemmatizer/lookups/lookups.bin  
  inflating: Spacy_Start_CUST_Model_with_emails_V2.08/meta.json  
   creating: Spacy_Start_CUST_Model_with_emails_V2.08/ner/
  inflating: Spacy_Start_CUST_Model_with_emails_V2.08/ner/cfg  
  inflating: Spacy_Start_CUST_Model_with_emails_V2.08/ner/model  
  inflating: Spacy_Start_CUST_Model_with_emails_V2.08/ner/moves  
   creating: Spacy_Start_CUST_Model_with_emails_V2.08/parser/
  inflating: Spacy_Start_CUST_Model_with_emails_V2.08/parser/cfg  
  inflating: Spacy_Start_CUST_Model_with_emails_V2.0

In [ ]:
##It is the pretrained spacy model. Can Use it if needed
# !python -m spacy download en_core_web_sm


###Model Training

In [ ]:
import spacy,random
from spacy.training import Example
from google.colab import files
from spacy.util import minibatch, compounding
def train_spacy(data, iterations, nlp):  # <-- Add model as nlp parameter
    TRAIN_DATA = data
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe('ner', last=True)

    elif 'ner' in nlp.pipe_names:
      ner=nlp.get_pipe("ner")


    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
                ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    # pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']  #pipe != 'ner'
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(4.0, 32.0, 1.001)
        # optimizer = nlp.begin_training()
        optimizer = nlp.create_optimizer()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
#                 text, annotations = zip(*batch)

#                 print(text, annotations)
#                 print(",".join(text))
                for text, annotations in batch:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
    #                 example = Example.from_dict(doc, {"entities": annotations})
                    nlp.update(
                        [example],  # batch of texts
                           # batch of annotations
                        drop=0.5,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                print(losses, 'Iteration Number: '+ str(itn))
    return nlp

# nlp = spacy.blank('en')  # create blank Language class  # Train new model.
nlp = spacy.load("en_core_web_sm") #Retrain the old model.
# nlp = spacy.load('en_core_web_sm')
nlp.max_length = 150000000000000
start_training = train_spacy(train_data, 50, nlp)
start_training.to_disk("Spacy_CUST_Model_with_emails_V2")



##Load the default spacy model
# spacy.load('en_core_web_sm')

#Code to zip the model as folder
# !zip -r /content/Company_Cust_Email_Model_100epochs.zip /content/Spacy_CUST_Model_with_emails_V2

# files.download("/content/Company_Cust_Email_Model_100epochs.zip")


In [ ]:
print(nlp.pipe_names)  # should include "ner"
print(nlp.get_pipe("ner").labels)

###Testing the trained Model

In [51]:
# !zip -r /content/file.zip /content/Spacy_CUST_NAME_Model
# start_training
# npl = spacy.load('en_core_web_sm')
import spacy
from spacy import displacy
from collections import Counter
colors = {"REFERENCE": "#ee8d48", "VESS": "#2988DC", "NAME":"#6EB071", "EMAIL":"#E05584", "PHONE":"#7F7521"}
options = {"colors": colors}
nlp2 = spacy.load('/content/Spacy_CUST_Model_with_emails')

a = "-----Original  Message-----\r\n\r\nDear Sir/Madam,\r\nAs per above reference please find attached Vessel Name: PACIFIC XXXX  reply back to us with the respective information.  \
\r\nOur reference:  PRN-20253A11111-B(01)\r\nShould you have any questions or comments to this email and/or the attached document,\
 please contact:\r\nRohit Sharma\r\n\
 Email address: abcrobot@company.com\r\nPhone number:  +91 99999 84731"

displacy.render(nlp2(str(a)), jupyter=True, style='ent', options=options)